# Winner prediction (manual labels)
In our dataset, we have multiple candidate, of which one gets selected as a winner. My goal in this section is to predict this winner based on the manually labelled data.

In [1]:
from util import get_preprocessed_dataset

df = get_preprocessed_dataset()
df[df["Test"] == 1]

,Test,Headline ID,Winner,Headline,Actief,Lang,Vragen,Interpunctie,Tweeledigheid,Emotie,Voorwaartse Verwijzing,Signaalwoorden,Lidwoorden,Adjectieven,Eigennamen,Betrekking,Voor+Achternaam,Cijfers,Quotes
0,1,A,False,Barack en Michelle Obama laten dansmoves zien ...,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,B,True,Barack en Michelle Obama gaan helemaal los tij...,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0


The number of candidate headlines per test is often different, e.g.:

In [2]:
df[df["Test"] == 16]

,Test,Headline ID,Winner,Headline,Actief,Lang,Vragen,Interpunctie,Tweeledigheid,Emotie,Voorwaartse Verwijzing,Signaalwoorden,Lidwoorden,Adjectieven,Eigennamen,Betrekking,Voor+Achternaam,Cijfers,Quotes
37,16,A,False,"Verpleegster klapt uit de biecht: ""Ja, onze pa...",1,1,0,0,1,0,0,0,0,0,0,0,0,0,1
38,16,B,False,"Verpleegster klapt uit de biecht: ""Ik kijk pat...",1,1,0,0,1,0,0,0,0,0,0,0,0,0,1
39,16,C,False,"Verpleegster klapt uit de biecht: ""Ik kijk hen...",1,1,0,0,1,0,0,0,0,0,0,0,0,0,1
40,16,D,False,"Verpleegster klapt uit de biecht: ""Soms is het...",1,1,0,0,1,0,0,0,0,0,0,0,0,0,1
41,16,E,True,"Verpleegster klapt uit de biecht: ""Ook wij voe...",1,1,0,0,1,0,0,0,0,0,0,0,0,0,1
42,16,F,False,"Verpleegster klapt uit de biecht: ""Ik gooide k...",1,1,0,0,1,0,0,0,0,0,0,0,0,0,1
43,16,G,False,"Verpleegster klapt uit de biecht., ""Eén keer g...",1,1,0,0,1,0,0,0,0,0,0,0,0,1,1
44,16,H,False,"Verpleegster klapt uit de biecht: ""Ik was 25 e...",1,1,0,0,1,0,0,0,0,0,0,0,0,1,1


In this set, you can already see that only the features that were manually labelled won't be enough to predict the winner.

In [3]:
print(
    f"There are {len(df.Test.unique())} tests in total, with on average {len(df.Test) / len(df.Test.unique()):.2f} candidate headlines per test.")

There are 909 tests in total, with on average 2.48 candidate headlines per test.


## Train-test split
Our train-test split is of course different than for our label classification problem. We now need to have all candidate headlines for a certain test in the same split, instead of only one candidate per test in each of them.

In [4]:
from sklearn.model_selection import train_test_split

# Get the unique test IDs
test_ids = df.Test.unique()

# Create a train test split based on the test IDs
train_ids, test_ids = train_test_split(test_ids, test_size=0.2, random_state=42)
train_ids[:5]

array([84, 11, 617, 250, 870], dtype=object)

In [5]:
# Get the train and test dataframes
train_df = df[df["Test"].isin(train_ids)]
test_df = df[df["Test"].isin(test_ids)]
train_df.head()

,Test,Headline ID,Winner,Headline,Actief,Lang,Vragen,Interpunctie,Tweeledigheid,Emotie,Voorwaartse Verwijzing,Signaalwoorden,Lidwoorden,Adjectieven,Eigennamen,Betrekking,Voor+Achternaam,Cijfers,Quotes
0,1,A,False,Barack en Michelle Obama laten dansmoves zien ...,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,B,True,Barack en Michelle Obama gaan helemaal los tij...,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,2,A,True,Marc Coucke maakt opvallende keuze bij start v...,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0
3,2,B,False,"Marc Coucke: ""Spelers van 10 miljoen? Neen, li...",1,0,0,0,1,0,0,0,0,0,0,0,1,0,1
4,3,A,True,Maandag drukste dag van het jaar op Brussels A...,1,1,0,0,1,0,0,0,0,0,0,1,0,0,1


In [6]:
test_df.head()

,Test,Headline ID,Winner,Headline,Actief,Lang,Vragen,Interpunctie,Tweeledigheid,Emotie,Voorwaartse Verwijzing,Signaalwoorden,Lidwoorden,Adjectieven,Eigennamen,Betrekking,Voor+Achternaam,Cijfers,Quotes
11,6,A,True,Cafébazin krijgt GAS-boete van 350 euro omdat ...,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1
12,6,B,False,Cafébazin krijgt GAS-boete van 350 euro omdat ...,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0
61,24,A,True,"Zaakvoerder betrapt winkeldief, en niet alleen...",1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
62,24,B,False,Betrapte dievegge in winterjas staat plots op ...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
65,26,A,False,"Knuffel is op zoek naar verloren kindje: ""Mijn...",1,1,0,0,1,1,0,0,0,0,0,0,0,0,1


In [7]:
# Get the number of unique values in "Test" column
print(f"{len(train_df.Test.unique())} tests in the train set, {len(test_df.Test.unique())} tests in the test set")

727 tests in the train set, 182 tests in the test set


In [8]:
# Split into train_x (containing all columns except Winner) and train_y (containing the Winner column)
train_x = train_df.drop(columns=["Winner"])
train_y = train_df[train_df["Winner"] == True][["Test", "Headline ID"]]

test_x = test_df.drop(columns=["Winner"])
test_y = test_df[test_df["Winner"] == True][["Test", "Headline ID"]]

In [9]:
train_x.head()

,Test,Headline ID,Headline,Actief,Lang,Vragen,Interpunctie,Tweeledigheid,Emotie,Voorwaartse Verwijzing,Signaalwoorden,Lidwoorden,Adjectieven,Eigennamen,Betrekking,Voor+Achternaam,Cijfers,Quotes
0,1,A,Barack en Michelle Obama laten dansmoves zien ...,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,B,Barack en Michelle Obama gaan helemaal los tij...,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,2,A,Marc Coucke maakt opvallende keuze bij start v...,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0
3,2,B,"Marc Coucke: ""Spelers van 10 miljoen? Neen, li...",1,0,0,0,1,0,0,0,0,0,0,0,1,0,1
4,3,A,Maandag drukste dag van het jaar op Brussels A...,1,1,0,0,1,0,0,0,0,0,0,1,0,0,1


In [10]:
train_y.head()

,Test,Headline ID
1,1,B
2,2,A
4,3,A
8,4,B
10,5,B


In [11]:
# Assert no duplicate entries in train_y Test column
assert len(train_y.Test.unique()) == len(train_y)

Only the test and the headline ID is the prefered format for the test dataset, but for the training dataset it might be useful to keep the Winner label, so it doesn't need to be reconstructed:

In [12]:
train_y_full = train_df[train_df["Winner"] == True][["Test", "Headline ID", "Winner"]]
train_y_full.head()

,Test,Headline ID,Winner
1,1,B,True
2,2,A,True
4,3,A,True
8,4,B,True
10,5,B,True


### Feature vectors
Often classifiers will only want the features itself as training set, instead of the entire dataframe, so let's also provide something for that.

In [13]:
from util import get_label_columns
train_x_features = train_x[get_label_columns()]
train_x_features.head()

,Actief,Lang,Vragen,Interpunctie,Tweeledigheid,Emotie,Voorwaartse Verwijzing,Signaalwoorden,Lidwoorden,Adjectieven,Eigennamen,Betrekking,Voor+Achternaam,Cijfers,Quotes
0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0
3,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1
4,1,1,0,0,1,0,0,0,0,0,0,1,0,0,1


## Metrics
Remind that this is our test_y:

In [31]:
test_y.head()

,Test,Headline ID
0,6,A
1,24,A
2,26,C
3,31,B
4,32,B


For our predictions, we'll also generate a list of candidates, let's create a random prediction by picking a random candidate for each test:

In [32]:
random_predictions = test_x.groupby("Test").apply(lambda x: x.sample(1, random_state=42))[["Test", "Headline ID"]]
random_predictions.head()

,,Test,Headline ID
Test,,,
6,12,6,B
24,62,24,B
26,65,26,A
31,78,31,B
32,80,32,B


In [33]:
# Merge the random predictions with the test_y (change "Headline ID" to "Predicted Headline ID" for random_predictions)
# random_predictions_renamed = random_predictions.rename(columns={"Headline ID": "Predicted Headline ID"}, inplace=True)
import pandas as pd

# Reset indices to avoid ambiguous index/column level problems
# src: https://stackoverflow.com/questions/67686902/valueerror-movieid-is-both-an-index-level-and-a-column-label-which-is-ambigu
test_y.reset_index(drop = True, inplace = True)
random_predictions.reset_index(drop = True, inplace = True)

test_y_pred = pd.merge(test_y, random_predictions, on=["Test"], suffixes=("", " Predicted"))
test_y_pred.head()

,Test,Headline ID,Headline ID Predicted
0,6,A,B
1,24,A,B
2,26,C,A
3,31,B,B
4,32,B,B


In [34]:
# Calculate the accuracy of the random predictions
from sklearn.metrics import accuracy_score
score = accuracy_score(test_y_pred["Headline ID"], test_y_pred["Headline ID Predicted"])
print(f"Random predictions accuracy: {100*score:.2f}%")

Random predictions accuracy: 54.14%


This accuracy is actually higher than I would've expected, since there are often more than two candidate headlines.

Accuracy is a good metric to use for this problem, since we won't have problems with data imbalance (since we'll pick only one winner instead of classifying each candidate as winner or not).